<a href="https://colab.research.google.com/github/IA2021UR/trabajo-ia-en-la-comunidad-de-la-rioja-equipo-6/blob/main/PruebaModeloRioja2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install semtorch

     |████████████████████████████████| 51kB 4.4MB/s 
     |████████████████████████████████| 29.2MB 134kB/s 
     |████████████████████████████████| 204kB 54.1MB/s 
     |████████████████████████████████| 61kB 7.8MB/s 
  Created wheel for semtorch: filename=SemTorch-0.1.1-cp37-none-any.whl size=50551 sha256=ffefb57aa291e2f7002450409088aad625065517bfeb3bb4ce34de5bccd141c4
  Stored in directory: /root/.cache/pip/wheels/f6/da/9f/726f9d2f054d71afa59afeca3f4672464001e9e3f2924d959e
Successfully built semtorch
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scikit-image 0.16.2
    Uninstalling scikit-image-0.16.2:
      Successfully uninstalled scikit-image-0.16.2
  Found existing installation: fastai 1.0.61
    Uninstalling fastai-1.0.61:
      Successfully uninstalled fastai-1.0.61


In [2]:
!pip install fastai --upgrade

Requirement already up-to-date: fastai in /usr/local/lib/python3.7/dist-packages (2.3.1)


In [3]:
from fastai.basics import *
from fastai.vision import models
from fastai.vision.all import *
from fastai.metrics import *
from fastai.data.all import *
from fastai.callback import *


from pathlib import Path
import random
from semtorch import get_segmentation_learner

/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [4]:
!wget https://www.dropbox.com/s/78tojo8s8j70xbb/dataset_prueba_RIOJA.zip?dl=0 -O dataset.zip

--2021-05-29 09:09:26--  https://www.dropbox.com/s/78tojo8s8j70xbb/dataset_prueba_RIOJA.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6032:18::a27d:5212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/78tojo8s8j70xbb/dataset_prueba_RIOJA.zip [following]
--2021-05-29 09:09:26--  https://www.dropbox.com/s/raw/78tojo8s8j70xbb/dataset_prueba_RIOJA.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc4b57dbf26390ef922be6a03bff.dl.dropboxusercontent.com/cd/0/inline/BPb4uuWMPBX63vcVJsKryeg-TNXRbCV-eg0QXgITi_ZY6sjrMo-WqJCbDpItOKt3oKSrIvhUdjUkO0Rm7lfK5A5Bdhfv5J-QrC_EhOIsd5wd37056N5ejIhDbaB0BA6WMXFh-fYORlXyP_csrfgpILld/file# [following]
--2021-05-29 09:09:26--  https://uc4b57dbf26390ef922be6a03bff.dl.dropboxusercontent.com/cd/0/inline/BPb4uuWMPBX63vcVJsKryeg-TNXRbCV-eg0QXgITi_ZY6sjr

In [5]:
!unzip dataset.zip

Archive:  dataset.zip
   creating: dataset_prueba_RIOJA/
 extracting: dataset_prueba_RIOJA/codes.txt  
   creating: dataset_prueba_RIOJA/Images/
   creating: dataset_prueba_RIOJA/Images/test/
 extracting: dataset_prueba_RIOJA/Images/test/ortofoto_ortofoto-2014_182082.jpg  
   creating: dataset_prueba_RIOJA/Images/train/
  inflating: dataset_prueba_RIOJA/Images/train/22678915_15.tiff  
  inflating: dataset_prueba_RIOJA/Images/train/22678930_15.tiff  
  inflating: dataset_prueba_RIOJA/Images/train/22678945_15.tiff  
  inflating: dataset_prueba_RIOJA/Images/train/22678960_15.tiff  
  inflating: dataset_prueba_RIOJA/Images/train/22678975_15.tiff  
  inflating: dataset_prueba_RIOJA/Images/train/22678990_15.tiff  
  inflating: dataset_prueba_RIOJA/Images/train/22679005_15.tiff  
  inflating: dataset_prueba_RIOJA/Images/train/22679020_15.tiff  
   creating: dataset_prueba_RIOJA/Images/val/
  inflating: dataset_prueba_RIOJA/Images/val/22978945_15.tiff  
  inflating: dataset_prueba_RIOJA/Images

In [6]:
path=Path('dataset_prueba_RIOJA/')

In [7]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [8]:
imagen = cv2.imread('dataset_prueba_RIOJA/Images/test/foto1.jpg', cv2.IMREAD_UNCHANGED)
dsize = (1500, 1500)
output = cv2.resize(imagen, dsize)
cv2.imwrite('dataset_prueba_RIOJA/Images/test/foto1.jpg', output) 

True

In [9]:
output.shape

(1500, 1500, 3)

In [10]:
import os

def recortar_en_trozos(imagen_ruta, altoyancho):
  imagen = cv2.imread(imagen_ruta)
  nombre = imagen_ruta.split('/')[3]
  carpeta2 = imagen_ruta.split('/')[2]
  carpeta1 = imagen_ruta.split('/')[1]
  extension = nombre.split('.')
  ancho = altoyancho
  alto = altoyancho
  i = 0
  for y in range(0,1500,300):
    for x in range(0,1500,300):
      i = i + 1
      recorte = imagen[y:y+altoyancho, x:x+altoyancho]
      cv2.imwrite('dataset_prueba_RIOJA/'+ carpeta1 + '/' + carpeta2 +'/' + extension[0] + '_' + str(i) + '.'+ extension[1], recorte)
  
  os.remove(imagen_ruta)

In [11]:
recortar_en_trozos('dataset_prueba_RIOJA/Images/test/foto1.jpg', 300)